In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ChiLit_Topic_Modeling

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ChiLit_Topic_Modeling


In [ ]:
!pip install git+https://github.com/tonazzog/OCTIS.git
!pip install optuna

In [1]:
import octis
from octis.models.LDA import LDA
from octis.models.ProdLDA import ProdLDA
from octis.models.ETM import ETM
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real
from skopt.space.space import Integer
from skopt.space.space import Categorical
import optuna
from typing import Tuple, List
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.io import show
import json
import os
import pickle

Note: OCTIS extension for multi-objective optimization doesn't work. OPTUNA was used for multi-objective optimization

In [2]:
chunk_size = 200
octis_folder = f"./octis_{chunk_size}/"
optuna_folder = f"./optuna_{chunk_size}/"

### Create OCTIS dataset

In [3]:
def prepare_octis_corpus(output_folder, docs):
  # Write to docs.tsv
  with open(os.path.join(output_folder, "corpus.tsv"), "w", encoding="utf-8") as f:
      for doc in docs:
          f.write(f"{doc}\n")
  # Tokenize and create vocabulary
  vocab = set()
  for doc in docs:
      vocab.update(doc.split())

  vocab = {w for w in vocab if w.isalpha() and len(w) > 2}
  vocab = sorted(vocab)  # sorting is optional but nice for consistency

  # Save vocab.json
  with open(os.path.join(output_folder, "vocab.json"), "w", encoding="utf-8") as f:
      json.dump(vocab, f)

In [4]:
df_chilit = pd.read_csv(f"./data/ChiLit_Chunks_{chunk_size}.csv")
#df_chilit = pd.read_csv(f"./data/ChiLit_Paragraphs.csv")
df_chilit = df_chilit.fillna("")
#df_chilit = df_chilit[df_chilit['tokens'] != '']

In [ ]:
df_chilit.head()

,book_id,chapter_num,paragraph_num,paragraph_text,tokens,num_tokens
1,alice,1,2,Alice was beginning to get very tired of sitti...,begin get tired sit sister bank have do peep b...,20
2,alice,1,3,So she was considering in her own mind (as wel...,consider own mind hot day make feel sleepy stu...,23
3,alice,1,4,There was nothing so VERY remarkable in that; ...,be remarkable think way hear say dear dear lat...,41
6,alice,1,7,"Either the well was very deep, or she fell ver...",deep fall have plenty time go look wonder go h...,46
8,alice,1,9,"Down, down, down. Would the fall NEVER come to...",fall come end wonder many mile fall time say g...,43


In [5]:
docs = []
for _, row in df_chilit.iterrows():
  docs.append(row["tokens"])

prepare_octis_corpus(octis_folder, docs)

#### Load OCTIS Dataset

In [6]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(octis_folder)

### Custom metric for OCTIS optimizer

In [ ]:
from octis.evaluation_metrics.metrics import AbstractMetric

class CoherenceDiversityCombination(AbstractMetric):
    def __init__(self, dataset, alpha=0.7):
        """
        alpha: weight for coherence (between 0 and 1).
        (1 - alpha) will be the weight for diversity.
        """
        super().__init__()
        self.alpha = alpha
        # Initialize base metrics
        self.c_v = Coherence(texts=dataset.get_corpus(), measure="c_npmi")
        self.diversity = TopicDiversity(topk=10)

    def score(self, model_output, **kwargs):
        # Compute individual scores
        coherence_score = self.c_v.score(model_output)
        diversity_score = self.diversity.score(model_output)

        # Weighted combination
        combined_score = (self.alpha * coherence_score + (1 - self.alpha) * diversity_score)

        return combined_score

## LDA model

### OCTIS LDA model optimization

In [ ]:
# Define evaluation metric
eval_metric = CoherenceDiversityCombination(dataset) # Initialize metric

# Istantiate model
model = LDA()

# Define the search space.
search_space = {
    "num_topics": Integer(10, 50),
    "alpha": Real(low=0.01, high=2.0),
    "eta": Real(low=0.0, high=2.0),
    "iterations": Integer(50, 200)
}

# Initialize an optimizer object and start the optimization.
optimizer = Optimizer()
optResult = optimizer.optimize(
    model, dataset,
    eval_metric,
    search_space,
    save_path=octis_folder,
    save_name = 'OCTIS_LDA',
    number_of_call=50, # number of optimization iterations
    model_runs=5 # number of runs of the topic model
)

#save the results of th optimization in a csv file
optResult.save_to_csv("OCTIS_LDA.csv")

#### Train model with best parameters

In [85]:
results = json.load(open(f"{octis_folder}/OCTIS_LDA.json",'r'))
best_iter = results['f_val'].index(max(results['f_val']))
model = LDA(
    num_topics=results['x_iters']['num_topics'][best_iter],
    alpha= results['x_iters']['alpha'][best_iter],
    eta= results['x_iters']['eta'][best_iter],
    passes=20)

output = model.train_model(dataset)

pickle.dump(output, open(octis_folder + "OCTIS_LDA_output.pkl", "wb"))

### Optuna LDA multi-objective optimization

In [ ]:
optuna.delete_study(study_name="LDA_Study", storage=f"sqlite:///{optuna_folder}LDA_Study.db")

In [ ]:
def objectiveLDA(trial) -> Tuple[float, float]:

    # Define hyperparameters to optimize
    num_topics = trial.suggest_int("num_topics", 10, 50)
    alpha = trial.suggest_float("alpha", 0.01, 0.2)
    eta = trial.suggest_float("eta", 0.01, 0.2)
    passes = trial.suggest_int("passes", 5, 30)
    iterations = trial.suggest_int("iterations", 50, 200)

    # Train LDA model
    model = LDA(
        num_topics=num_topics,
        alpha=alpha,
        eta=eta,
        passes=passes,
        iterations=iterations,
        random_state=42,
    )

    output = model.train_model(dataset)

    # Compute coherence score
    coherence_metrics = Coherence(texts=dataset.get_corpus(), #list of our documents
                    measure='c_npmi')
    coherence = coherence_metrics.score(output)

    # Compute diversity score
    diverisity_metric = TopicDiversity(topk=10) # Initialize metric
    diversity = diverisity_metric.score(output)

    return coherence, diversity  # Optuna will maximize these


# Run optimization
study = optuna.create_study(
    directions=["maximize","maximize"],
    storage=f"sqlite:///{optuna_folder}LDA_Study.db",
    study_name="LDA_Study")
study.optimize(objectiveLDA, n_trials=50)

[I 2025-08-12 23:35:10,519] A new study created in RDB with name: LDA_Study
[I 2025-08-12 23:40:50,226] Trial 0 finished with values: [-0.01527873111567863, 0.74] and parameters: {'num_topics': 25, 'alpha': 0.040399725923978266, 'eta': 0.012447161904934178, 'passes': 10, 'iterations': 195}.
[I 2025-08-12 23:46:25,126] Trial 1 finished with values: [-0.1409386265621207, 0.9129032258064517] and parameters: {'num_topics': 31, 'alpha': 0.19711802101251766, 'eta': 0.13859233231342827, 'passes': 26, 'iterations': 153}.
[I 2025-08-12 23:49:15,992] Trial 2 finished with values: [-0.06718309715057955, 0.8076923076923077] and parameters: {'num_topics': 13, 'alpha': 0.02329181016923864, 'eta': 0.1422413420782142, 'passes': 12, 'iterations': 176}.
[I 2025-08-12 23:56:14,521] Trial 3 finished with values: [-0.061582172735959735, 0.8377777777777777] and parameters: {'num_topics': 45, 'alpha': 0.18177054258786077, 'eta': 0.03887815152676666, 'passes': 21, 'iterations': 126}.
[I 2025-08-12 23:59:36,81

In [101]:
def train_final_LDA_model(params):
    """Train final LDA model with selected parameters"""
    print(f"\nTraining final model with parameters: {params}")

    model = LDA(
        num_topics=params["num_topics"],
        alpha=params["alpha"],
        eta=params["eta"],
        passes=params["passes"],
        iterations=params["iterations"],
        random_state=42,
    )

    output = model.train_model(dataset)

    # Calculate final metrics
    coherence_metrics = Coherence(texts=dataset.get_corpus(), #list of our documents
                    measure='c_npmi')
    coherence = coherence_metrics.score(output)

    diverisity_metric = TopicDiversity(topk=10) # Initialize metric
    diversity = diverisity_metric.score(output)

    print(f"Final model metrics:")
    print(f"  Coherence: {coherence:.4f}")
    print(f"  Diversity: {diversity:.4f}")

    return model, output, coherence, diversity

In [102]:
LDA_study = optuna.load_study(
    storage=f"sqlite:///{optuna_folder}LDA_Study.db",
    study_name="LDA_Study"
)

In [103]:
if LDA_study.best_trials:
    # Get balanced solution
    pareto_trials = LDA_study.best_trials

    # Pick the first Pareto optimal solution
    selected_params = pareto_trials[0].params
    final_model, final_output, final_coherence, final_diversity = train_final_LDA_model(selected_params)

    print(f"\nFinal model trained successfully!")

pickle.dump(final_output, open(optuna_folder + "Optuna_LDA_output.pkl", "wb"))


Training final model with parameters: {'num_topics': 14, 'alpha': 0.08263819021129706, 'eta': 0.11478421778029362, 'passes': 29, 'iterations': 139}
Final model metrics:
  Coherence: -0.0220
  Diversity: 0.8214

Final model trained successfully!


## ProdLDA

### OCTIS ProdLDA optimization

In [4]:
# Initialize metric
eval_metric = CoherenceDiversityCombination(dataset) # Initialize metric

# Initialize model
model = ProdLDA(use_partitions=False)

# Define the search space.
search_space = {
    "num_topics": Integer(10,50),
    "dropout" : Real(low=0, high=0.60),
    "num_neurons" : Categorical([50, 100, 200, 300]),
    "num_layers": Integer(1,3),
    "activation" : Categorical(["softplus","relu", "sigmoid"])
}

# Initialize an optimizer object and start the optimization.
optimizer=Optimizer()
optResult=optimizer.optimize(
    model, dataset,
    eval_metric,
    search_space,
    save_name='OCTIS_ProdLDA',
    save_path=octis_folder,
    number_of_call=50, # number of optimization iterations
    model_runs=5  # number of runs of the topic model
)

#save the results of th optimization in a csv file
optResult.save_to_csv("OCTIS_ProdLDA.csv")

NameError: name 'CoherenceDiversityCombination' is not defined

### Train ProdLDA model with best parameters

In [97]:
results = json.load(open(f"{octis_folder}/OCTIS_ProdLDA.json",'r'))
best_iter = results['f_val'].index(max(results['f_val']))

model = ProdLDA(
    num_topics=results['x_iters']['num_topics'][best_iter],
    dropout=results['x_iters']['dropout'][best_iter],
    num_neurons=results['x_iters']['num_neurons'][best_iter],
    num_layers=results['x_iters']['num_layers'][best_iter],
    activation=results['x_iters']['activation'][best_iter],
    use_partitions=False
)

output = model.train_model(dataset)

pickle.dump(output, open(octis_folder + "OCTIS_ProdLDA_output.pkl", "wb"))

Epoch: [1/100]	Samples: [5843/584300]	Train Loss: 2257.5753472370784	Time: 0:00:04.272165
Epoch: [2/100]	Samples: [11686/584300]	Train Loss: 2216.680636819699	Time: 0:00:04.100038
Epoch: [3/100]	Samples: [17529/584300]	Train Loss: 2188.057128989817	Time: 0:00:04.197414
Epoch: [4/100]	Samples: [23372/584300]	Train Loss: 2171.4995895195107	Time: 0:00:03.960664
Epoch: [5/100]	Samples: [29215/584300]	Train Loss: 2159.9728153613296	Time: 0:00:03.670171
Epoch: [6/100]	Samples: [35058/584300]	Train Loss: 2152.416890403046	Time: 0:00:04.057952
Epoch: [7/100]	Samples: [40901/584300]	Train Loss: 2146.714863137515	Time: 0:00:03.619083
Epoch: [8/100]	Samples: [46744/584300]	Train Loss: 2141.3794129995294	Time: 0:00:03.998018
Epoch: [9/100]	Samples: [52587/584300]	Train Loss: 2139.1175999860943	Time: 0:00:04.149957
Epoch: [10/100]	Samples: [58430/584300]	Train Loss: 2137.014820752396	Time: 0:00:04.178687
Epoch: [11/100]	Samples: [64273/584300]	Train Loss: 2133.2107302541503	Time: 0:00:04.289923
Epo

### Optuna ProdLDA multi-objective optimization

In [ ]:
def objectiveProdLDA(trial) -> Tuple[float, float]:
    # Define hyperparameters to optimize
    num_topics = trial.suggest_int("num_topics", 10, 50)
    dropout = trial.suggest_float("dropout", 0, 0.60)
    num_neurons = trial.suggest_categorical("num_neurons", [50, 100, 200, 300])
    num_layers = trial.suggest_int("num_layers", 1, 2)
    activation = trial.suggest_categorical("activation", ["softplus","relu", "sigmoid"])


    # Train ProdLDA model
    model = ProdLDA(
        num_topics=num_topics,
        dropout = dropout,
        num_neurons=num_neurons,
        num_layers=num_layers,
        activation=activation,
        use_partitions=False
    )

    output = model.train_model(dataset)

    # Compute coherence score (can also use perplexity, but coherence is often better)
    coherence_metrics = Coherence(texts=dataset.get_corpus(), #list of our documents
                    measure='c_npmi')
    coherence = coherence_metrics.score(output)

    diverisity_metric = TopicDiversity(topk=10) # Initialize metric
    diversity = diverisity_metric.score(output)

    return coherence, diversity  # Optuna will maximize these

# Run optimization
study = optuna.create_study(
    directions=["maximize","maximize"],
    storage=f"sqlite:///{optuna_folder}ProdLDA_Study.db",
    study_name="ProdLDA_Study"
  )
study.optimize(objectiveProdLDA, n_trials=50)

[I 2025-08-13 22:03:02,934] A new study created in RDB with name: ProdLDA_Study


Epoch: [1/100]	Samples: [26586/2658600]	Train Loss: 422.110667349298	Time: 0:00:24.608389
Epoch: [2/100]	Samples: [53172/2658600]	Train Loss: 410.1606030605864	Time: 0:00:21.659467
Epoch: [3/100]	Samples: [79758/2658600]	Train Loss: 406.31787976255123	Time: 0:00:21.464043
Epoch: [4/100]	Samples: [106344/2658600]	Train Loss: 404.4543563738316	Time: 0:00:23.315591
Epoch: [5/100]	Samples: [132930/2658600]	Train Loss: 403.264892229169	Time: 0:00:22.471757
Epoch: [6/100]	Samples: [159516/2658600]	Train Loss: 402.3959376895382	Time: 0:00:21.106510
Epoch: [7/100]	Samples: [186102/2658600]	Train Loss: 402.0111157540623	Time: 0:00:22.316645
Epoch: [8/100]	Samples: [212688/2658600]	Train Loss: 401.4959139825707	Time: 0:00:22.660983
Epoch: [9/100]	Samples: [239274/2658600]	Train Loss: 401.0708883141033	Time: 0:00:22.463995
Epoch: [10/100]	Samples: [265860/2658600]	Train Loss: 400.4433144515088	Time: 0:00:22.273739
Epoch: [11/100]	Samples: [292446/2658600]	Train Loss: 400.4027887974404	Time: 0:00:

[I 2025-08-13 22:40:46,735] Trial 0 finished with values: [0.030310650198585733, 0.9842105263157894] and parameters: {'num_topics': 19, 'dropout': 0.20138708349811507, 'num_neurons': 300, 'num_layers': 1, 'activation': 'sigmoid'}.


Epoch: [1/100]	Samples: [26586/2658600]	Train Loss: 424.3356337613664	Time: 0:00:14.580425
Epoch: [2/100]	Samples: [53172/2658600]	Train Loss: 415.51297521193015	Time: 0:00:11.899327
Epoch: [3/100]	Samples: [79758/2658600]	Train Loss: 413.02796595688284	Time: 0:00:12.639376
Epoch: [4/100]	Samples: [106344/2658600]	Train Loss: 411.6510305368074	Time: 0:00:12.408113
Epoch: [5/100]	Samples: [132930/2658600]	Train Loss: 410.8688356374713	Time: 0:00:12.876275
Epoch: [6/100]	Samples: [159516/2658600]	Train Loss: 410.2421379482505	Time: 0:00:12.397923
Epoch: [7/100]	Samples: [186102/2658600]	Train Loss: 409.52713239287124	Time: 0:00:12.313817
Epoch: [8/100]	Samples: [212688/2658600]	Train Loss: 409.19875280193287	Time: 0:00:12.396397
Epoch: [9/100]	Samples: [239274/2658600]	Train Loss: 408.9437831177593	Time: 0:00:11.805877
Epoch: [10/100]	Samples: [265860/2658600]	Train Loss: 408.59055525288215	Time: 0:00:12.740643
Epoch: [11/100]	Samples: [292446/2658600]	Train Loss: 408.29346191663376	Time

[I 2025-08-13 23:04:27,680] Trial 1 finished with values: [-0.003990636319932234, 0.915] and parameters: {'num_topics': 20, 'dropout': 0.45703562599863395, 'num_neurons': 50, 'num_layers': 1, 'activation': 'relu'}.


...

[I 2025-08-15 01:14:44,584] Trial 47 finished with values: [0.049236189061073056, 0.926829268292683] and parameters: {'num_topics': 41, 'dropout': 0.10669488347501321, 'num_neurons': 300, 'num_layers': 2, 'activation': 'softplus'}.


Epoch: [1/100]	Samples: [26586/2658600]	Train Loss: 428.86398595742355	Time: 0:00:15.323080
Epoch: [2/100]	Samples: [53172/2658600]	Train Loss: 416.15079493642094	Time: 0:00:14.055252
Epoch: [3/100]	Samples: [79758/2658600]	Train Loss: 412.21777751477515	Time: 0:00:13.896217
Epoch: [4/100]	Samples: [106344/2658600]	Train Loss: 410.3147031978767	Time: 0:00:14.059058
Epoch: [5/100]	Samples: [132930/2658600]	Train Loss: 409.14679439484127	Time: 0:00:14.031136
Epoch: [6/100]	Samples: [159516/2658600]	Train Loss: 408.5514023549166	Time: 0:00:13.927852
Epoch: [7/100]	Samples: [186102/2658600]	Train Loss: 408.0300839360354	Time: 0:00:14.302967
Epoch: [8/100]	Samples: [212688/2658600]	Train Loss: 407.63350251454006	Time: 0:00:13.895020
Epoch: [9/100]	Samples: [239274/2658600]	Train Loss: 407.32589307753614	Time: 0:00:14.059989
Epoch: [10/100]	Samples: [265860/2658600]	Train Loss: 406.98908506092255	Time: 0:00:13.991269
Epoch: [11/100]	Samples: [292446/2658600]	Train Loss: 406.7766473073234	Tim

[I 2025-08-15 01:45:57,669] Trial 48 finished with values: [0.028531295707150617, 0.94] and parameters: {'num_topics': 35, 'dropout': 0.2890307828033601, 'num_neurons': 100, 'num_layers': 2, 'activation': 'relu'}.


Epoch: [1/100]	Samples: [26586/2658600]	Train Loss: 439.27148580755613	Time: 0:00:23.348715
Epoch: [2/100]	Samples: [53172/2658600]	Train Loss: 422.55804996638267	Time: 0:00:20.757790
Epoch: [3/100]	Samples: [79758/2658600]	Train Loss: 417.0997653399581	Time: 0:00:20.569704
Epoch: [4/100]	Samples: [106344/2658600]	Train Loss: 414.5152610572764	Time: 0:00:19.985652
Epoch: [5/100]	Samples: [132930/2658600]	Train Loss: 413.02028337870496	Time: 0:00:20.143417
Epoch: [6/100]	Samples: [159516/2658600]	Train Loss: 411.68399269851847	Time: 0:00:26.785909
Epoch: [7/100]	Samples: [186102/2658600]	Train Loss: 410.820102501963	Time: 0:00:20.331504
Epoch: [8/100]	Samples: [212688/2658600]	Train Loss: 410.1682572080348	Time: 0:00:20.044223
Epoch: [9/100]	Samples: [239274/2658600]	Train Loss: 409.4866017432225	Time: 0:00:20.087074
Epoch: [10/100]	Samples: [265860/2658600]	Train Loss: 409.06440507927107	Time: 0:00:19.948703
Epoch: [11/100]	Samples: [292446/2658600]	Train Loss: 408.8392454190528	Time: 

[I 2025-08-15 02:23:40,577] Trial 49 finished with values: [0.026318954114752924, 0.9166666666666666] and parameters: {'num_topics': 42, 'dropout': 0.377587470715686, 'num_neurons': 300, 'num_layers': 1, 'activation': 'sigmoid'}.


In [8]:
def train_final_ProdLDA_model(params):
    """Train final LDA model with selected parameters"""
    print(f"\nTraining final model with parameters: {params}")

    model = ProdLDA(
        num_topics=params["num_topics"],
        dropout=params["dropout"],
        num_neurons=params["num_neurons"],
        num_layers=params["num_layers"],
        activation=params["activation"],
        use_partitions=False
    )

    output = model.train_model(dataset)

    # Calculate final metrics
    coherence_metrics = Coherence(texts=dataset.get_corpus(), #list of our documents
                    measure='c_npmi')
    coherence = coherence_metrics.score(output)

    diverisity_metric = TopicDiversity(topk=10) # Initialize metric
    diversity = diverisity_metric.score(output)

    print(f"Final model metrics:")
    print(f"  Coherence: {coherence:.4f}")
    print(f"  Diversity: {diversity:.4f}")

    return model, output, coherence, diversity

In [7]:
ProdLDA_study = optuna.load_study(
    storage=f"sqlite:///{optuna_folder}ProdLDA_Study.db",
    study_name="ProdLDA_Study"
)

In [9]:
if ProdLDA_study.best_trials:
    # Get balanced solution
    pareto_trials = ProdLDA_study.best_trials

    # Pick the first Pareto optimal solution
    selected_params = pareto_trials[0].params
    final_model, final_output, final_coherence, final_diversity = train_final_ProdLDA_model(selected_params)

    print(f"\nFinal model trained successfully!")

    pickle.dump(final_output, open(optuna_folder + "Optuna_ProdLDA_output.pkl", "wb"))


Training final model with parameters: {'num_topics': 18, 'dropout': 0.1373903093404273, 'num_neurons': 300, 'num_layers': 1, 'activation': 'sigmoid'}
Epoch: [1/100]	Samples: [5843/584300]	Train Loss: 2246.3884215086428	Time: 0:00:06.148821
Epoch: [2/100]	Samples: [11686/584300]	Train Loss: 2196.185287817688	Time: 0:00:04.758910
Epoch: [3/100]	Samples: [17529/584300]	Train Loss: 2170.5700049739003	Time: 0:00:04.917258
Epoch: [4/100]	Samples: [23372/584300]	Train Loss: 2154.904042497433	Time: 0:00:04.796963
Epoch: [5/100]	Samples: [29215/584300]	Train Loss: 2144.8061088054083	Time: 0:00:04.559612
Epoch: [6/100]	Samples: [35058/584300]	Train Loss: 2136.6208911838953	Time: 0:00:05.026892
Epoch: [7/100]	Samples: [40901/584300]	Train Loss: 2131.2892764311996	Time: 0:00:04.862225
Epoch: [8/100]	Samples: [46744/584300]	Train Loss: 2126.4075543117833	Time: 0:00:04.804730
Epoch: [9/100]	Samples: [52587/584300]	Train Loss: 2124.5415561355467	Time: 0:00:04.724329
Epoch: [10/100]	Samples: [58430/5

## ETM

### OCTIS ETM optimization

In [ ]:
# Initialize metric
eval_metric = CoherenceDiversityCombination(dataset) # Initialize metric

# Initialize odel
model = ETM(
    use_partitions=False,
    device='cuda',
    embeddings_path='./data/chilit-19th-century-averaged-embeddings.txt',
)

# Define the search space.
search_space = {
    "num_topics": Integer(10,50),
    "dropout" : Real(low=0, high=0.60),
    "t_hidden_size" : Categorical([50, 100, 200, 300]),
    "activation" : Categorical(["softplus", "relu", "sigmoid"])
}

# Initialize an optimizer object and start the optimization.
optimizer=Optimizer()
optResult=optimizer.optimize(
    model, dataset,
    eval_metric,
    search_space,
    save_name='OCTIS_ETM',
    save_path=octis_folder,
    number_of_call=50, # number of optimization iterations
    model_runs=5  # number of runs of the topic model
)

# Save the results of th optimization in a csv file
optResult.save_to_csv("OCTIS_ETM.csv")

Streaming output truncated to the last 5000 lines.
Epoch----->13 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 1726.29 .. NELBO: 1726.29
****************************************************************************************************
Epoch: 14 .. batch: 20/46 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 1714.74 .. NELBO: 1714.74
Epoch: 14 .. batch: 40/46 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 1723.1 .. NELBO: 1723.1
****************************************************************************************************
Epoch----->14 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 1726.23 .. NELBO: 1726.23
****************************************************************************************************
Epoch: 15 .. batch: 20/46 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 1714.52 .. NELBO: 1714.52
Epoch: 15 .. batch: 40/46 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 1722.99 .. NELBO: 1722.99
****************************************************************************************************
E

### Train ETM model with best parameters

In [105]:
results = json.load(open(f"{octis_folder}/OCTIS_ETM.json",'r'))
best_iter = results['f_val'].index(max(results['f_val']))
model = ETM(
    num_topics=results['x_iters']['num_topics'][best_iter],
    dropout=results['x_iters']['dropout'][best_iter],
    t_hidden_size=results['x_iters']['t_hidden_size'][best_iter],
    activation=results['x_iters']['activation'][best_iter],
    device = 'cuda',
    embeddings_path='./data/chilit-19th-century-averaged-embeddings.txt',
    use_partitions=False
)

output = model.train_model(dataset)
pickle.dump(output, open(octis_folder + "OCTIS_ETM_Output.pkl", "wb"))



model: ETM(
  (t_drop): Dropout(p=0.5920383989283502, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=23743, bias=False)
  (alphas): Linear(in_features=300, out_features=10, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=23743, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=300, out_features=10, bias=True)
  (logsigma_q_theta): Linear(in_features=300, out_features=10, bias=True)
)
Epoch: 1 .. batch: 20/46 .. LR: 0.005 .. KL_theta: 0.34 .. Rec_loss: 1991.97 .. NELBO: 1992.31
Epoch: 1 .. batch: 40/46 .. LR: 0.005 .. KL_theta: 0.5 .. Rec_loss: 1878.18 .. NELBO: 1878.68
****************************************************************************************************
Epoch----->1 .. LR: 0.005 .. KL_theta: 0.54 .. Rec_loss: 1866.79 .. NELBO: 1867.33
***************************************************************************

### Optuna ETM multi-objective optimization

In [ ]:
optuna.delete_study(study_name="ETM_Study", storage=f"sqlite:///{optuna_folder}ETM_Study.db")

In [ ]:
def objectiveETM(trial) -> Tuple[float, float]:

    # Define hyperparameters to optimize
    num_topics = trial.suggest_int("num_topics", 10, 50)
    dropout = trial.suggest_float("dropout", 0, 0.60)
    t_hidden_size = trial.suggest_categorical("t_hidden_size", [50, 100, 200, 300])
    activation = trial.suggest_categorical("activation", ["softplus", "relu", "sigmoid"])

    # Train ETM model
    model = ETM(
        num_topics=num_topics,
        dropout = dropout,
        t_hidden_size=t_hidden_size,
        activation=activation,
        device = 'cuda',
        embeddings_path='./data/chilit-19th-century-averaged-embeddings.txt',
        use_partitions=False,
        num_epochs = 50
    )

    output = model.train_model(dataset)

    # Compute coherence score (can also use perplexity, but coherence is often better)
    coherence_metrics = Coherence(texts=dataset.get_corpus(), #list of our documents
                    measure='c_npmi')
    coherence = coherence_metrics.score(output)

    diverisity_metric = TopicDiversity(topk=10) # Initialize metric
    diversity = diverisity_metric.score(output)

    return coherence, diversity  # Optuna will maximize these

# Run optimization
study = optuna.create_study(
    directions=["maximize","maximize"],
    storage=f"sqlite:///{octis_folder}ETM_Study.db",
    study_name="ETM_Study"
  )
study.optimize(objectiveETM, n_trials=50)

[I 2025-08-18 16:32:42,639] A new study created in RDB with name: ETM_Study


model: ETM(
  (t_drop): Dropout(p=0.5017876433740717, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=23743, bias=False)
  (alphas): Linear(in_features=300, out_features=26, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=23743, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=100, out_features=26, bias=True)
  (logsigma_q_theta): Linear(in_features=100, out_features=26, bias=True)
)
Epoch: 1 .. batch: 20/46 .. LR: 0.005 .. KL_theta: 0.31 .. Rec_loss: 2001.76 .. NELBO: 2002.07
Epoch: 1 .. batch: 40/46 .. LR: 0.005 .. KL_theta: 0.41 .. Rec_loss: 1883.95 .. NELBO: 1884.36
****************************************************************************************************
Epoch----->1 .. LR: 0.005 .. KL_theta: 0.42 .. Rec_loss: 1871.89 .. NELBO: 1872.31
**************************************************************************

[I 2025-08-18 16:34:00,834] Trial 0 finished with values: [-0.00842551799202895, 0.046153846153846156] and parameters: {'num_topics': 26, 'dropout': 0.5017876433740717, 't_hidden_size': 100, 'activation': 'relu'}.


model: ETM(
  (t_drop): Dropout(p=0.23344863649595227, inplace=False)
  (theta_act): Softplus(beta=1.0, threshold=20.0)
  (rho): Linear(in_features=300, out_features=23743, bias=False)
  (alphas): Linear(in_features=300, out_features=21, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=23743, out_features=300, bias=True)
    (1): Softplus(beta=1.0, threshold=20.0)
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): Softplus(beta=1.0, threshold=20.0)
  )
  (mu_q_theta): Linear(in_features=300, out_features=21, bias=True)
  (logsigma_q_theta): Linear(in_features=300, out_features=21, bias=True)
)
Epoch: 1 .. batch: 20/46 .. LR: 0.005 .. KL_theta: 1.76 .. Rec_loss: 2015.32 .. NELBO: 2017.08
Epoch: 1 .. batch: 40/46 .. LR: 0.005 .. KL_theta: 0.93 .. Rec_loss: 1891.9 .. NELBO: 1892.83
****************************************************************************************************
Epoch----->1 .. LR: 0.005 .. KL_theta: 0.83 .. Rec_loss: 1879.11 .. NELBO

[I 2025-08-18 16:35:15,249] Trial 1 finished with values: [-0.00816951656097918, 0.05238095238095238] and parameters: {'num_topics': 21, 'dropout': 0.23344863649595227, 't_hidden_size': 300, 'activation': 'softplus'}.


...

[I 2025-08-18 17:32:31,871] Trial 47 finished with values: [-0.007972072099875022, 0.09090909090909091] and parameters: {'num_topics': 11, 'dropout': 0.15699856688065783, 't_hidden_size': 200, 'activation': 'softplus'}.


model: ETM(
  (t_drop): Dropout(p=0.25320981797696485, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=23743, bias=False)
  (alphas): Linear(in_features=300, out_features=38, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=23743, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=300, out_features=38, bias=True)
  (logsigma_q_theta): Linear(in_features=300, out_features=38, bias=True)
)
Epoch: 1 .. batch: 20/46 .. LR: 0.005 .. KL_theta: 0.31 .. Rec_loss: 2005.89 .. NELBO: 2006.2
Epoch: 1 .. batch: 40/46 .. LR: 0.005 .. KL_theta: 0.28 .. Rec_loss: 1886.5 .. NELBO: 1886.78
****************************************************************************************************
Epoch----->1 .. LR: 0.005 .. KL_theta: 0.28 .. Rec_loss: 1874.19 .. NELBO: 1874.47
***************************************************************************

[I 2025-08-18 17:33:52,459] Trial 48 finished with values: [-0.00402310855691646, 0.11315789473684211] and parameters: {'num_topics': 38, 'dropout': 0.25320981797696485, 't_hidden_size': 300, 'activation': 'relu'}.


model: ETM(
  (t_drop): Dropout(p=0.447096182596987, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=23743, bias=False)
  (alphas): Linear(in_features=300, out_features=13, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=23743, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=300, out_features=13, bias=True)
  (logsigma_q_theta): Linear(in_features=300, out_features=13, bias=True)
)
Epoch: 1 .. batch: 20/46 .. LR: 0.005 .. KL_theta: 0.12 .. Rec_loss: 1993.05 .. NELBO: 1993.17
Epoch: 1 .. batch: 40/46 .. LR: 0.005 .. KL_theta: 0.28 .. Rec_loss: 1879.34 .. NELBO: 1879.62
****************************************************************************************************
Epoch----->1 .. LR: 0.005 .. KL_theta: 0.32 .. Rec_loss: 1867.82 .. NELBO: 1868.14
***************************************************************************

[I 2025-08-18 17:35:06,444] Trial 49 finished with values: [-0.009427572215897437, 0.2076923076923077] and parameters: {'num_topics': 13, 'dropout': 0.447096182596987, 't_hidden_size': 300, 'activation': 'relu'}.


In [95]:
def train_final_ETM_model(params):
    """Train final LDA model with selected parameters"""
    print(f"\nTraining final model with parameters: {params}")

    model = ETM(
        num_topics=params['num_topics'],
        dropout = params['dropout'],
        t_hidden_size=params['t_hidden_size'],
        activation=params['activation'],
        device = 'cuda',
        embeddings_path='./data/chilit-19th-century-averaged-embeddings.txt',
        use_partitions=False,
        num_epochs = 50
    )

    output = model.train_model(dataset)

    # Calculate final metrics
    coherence_metrics = Coherence(texts=dataset.get_corpus(), #list of our documents
                    measure='c_npmi')
    coherence = coherence_metrics.score(output)

    diverisity_metric = TopicDiversity(topk=10) # Initialize metric
    diversity = diverisity_metric.score(output)

    print(f"Final model metrics:")
    print(f"  Coherence: {coherence:.4f}")
    print(f"  Diversity: {diversity:.4f}")

    return model, output, coherence, diversity

In [94]:
ETM_study = optuna.load_study(
    storage=f"sqlite:///{optuna_folder}ETM_Study.db",
    study_name="ETM_Study"
)

In [96]:
if ETM_study.best_trials:
    # Get balanced solution
    pareto_trials = ETM_study.best_trials

    # Pick the first Pareto optimal solution
    selected_params = pareto_trials[0].params
    final_model, final_output, final_coherence, final_diversity = train_final_ETM_model(selected_params)

    print(f"\nFinal model trained successfully!")

    pickle.dump(final_output, open(optuna_folder + "Optuna_ETM_output.pkl", "wb"))



Training final model with parameters: {'num_topics': 11, 'dropout': 0.2769868853252398, 't_hidden_size': 300, 'activation': 'relu'}
model: ETM(
  (t_drop): Dropout(p=0.2769868853252398, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=23743, bias=False)
  (alphas): Linear(in_features=300, out_features=11, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=23743, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=300, out_features=11, bias=True)
  (logsigma_q_theta): Linear(in_features=300, out_features=11, bias=True)
)
Epoch: 1 .. batch: 20/46 .. LR: 0.005 .. KL_theta: 0.77 .. Rec_loss: 1990.77 .. NELBO: 1991.54
Epoch: 1 .. batch: 40/46 .. LR: 0.005 .. KL_theta: 0.82 .. Rec_loss: 1876.91 .. NELBO: 1877.73
****************************************************************************************************
Epoch----->1 .. LR: 0.00